In [39]:
import pandas as pd
import datetime as dt
data = pd.read_excel('/content/drive/MyDrive/WORKING/Работа /test_result.xlsx')
data = data[['Дата', 'ИНН Контрагента', 'Сумма транзакции', 'Record_ID']].copy()
data['Дата'] = data['Дата'].astype('datetime64')
data['splitter'] = 0
def insert_splitter(df):
    '''Функция на вход принимает сгруппированный датафрейм по одному из контрагентов
       и возращает датафрейм с меткой класса splitter.
       splitter = 1 - фрод
    '''
    # список уникальных records
    splita=[]
    records = df['Record_ID'].unique()
    for record in records:
        if record in splita:
          continue
        # создаем фильтр для датафрейма по Record_ID
        mask_record = df['Record_ID'] == record
        # определяем для Record_ID дату транзакции в виде серии Пандас
        date_record = df.loc[mask_record, 'Дата']
        # дата в представлении timestamp
        start_date = pd.to_datetime(date_record.values[0])
        # конечная дата для получение итоговой суммы транзакций
        end_date = start_date + dt.timedelta(days=30)
        # итоговая сумма транзакций
        total = df.loc[df['Дата'].between(start_date, end_date), 'Сумма транзакции'].sum()
        # проверка на фрод
        if total > 6000000:
            label_indxs = df[df['Дата'].between(start_date, end_date)].index
            df.loc[label_indxs, 'splitter'] = 1
        bufer=df.loc[df['splitter']==1]['Record_ID'].tolist()
        splita.append(bufer)
    return df
# Выделяем столбцы для формирования сгруппированного датафрейма
name_columns = ['ИНН Контрагента', 'Дата','Сумма транзакции', 'Record_ID', 'splitter']
# Группировка по контрагенту
agent_df = data.groupby(data['ИНН Контрагента'])[name_columns].apply(lambda x: insert_splitter(x))
agent_df.to_excel('/content/drive/MyDrive/WORKING/Работа /result.xlsx')

<ipython-input-39-7d494b4619a7>:5: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  data['Дата'] = data['Дата'].astype('datetime64')
<ipython-input-39-7d494b4619a7>:38: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  agent_df = data.groupby(data['ИНН Контрагента'])[name_columns].apply(lambda x: insert_splitter(x))
